# Lights Out Puzzles

https://www.reddit.com/r/puzzles/comments/1lrive7/what_is_this_type_of_puzzle_called_and_how_can_i/

https://en.wikipedia.org/wiki/Lights_Out_(game)

These puzzles can be written in terms of either solving (or maximizing a linear objective function) for a system of linear equations over a finite field.

https://en.wikipedia.org/wiki/Finite_field

## Transcription

In [ ]:
from PIL import Image, ImageDraw, ImageFont

In [ ]:
import numpy as np

In [ ]:
import pytesseract

In [ ]:
def transcribe_grid(img, n, margin=0, threshold=128, verbose=False):
    img = img.convert('L')
    arr = np.array(img)
    bin_arr = arr > threshold
    img = Image.fromarray(bin_arr)
    
    dx, dy = np.array(img.size) // n

    if verbose:
        print(f"{n} columns, spaced {dx}")
        print(f"{n} rows, spaced {dy}")

    grid = []
    for i in range(n):
        grid.append([])
        for j in range(n):
            cimg = img.crop((j * dx, i * dy, (j+1) * dx, (i+1) * dy))
            cimg = cimg.crop(np.array(cimg.size * 2) * np.array([0.1, 0.1, 1-margin, 1-margin]))
            c = pytesseract.image_to_string(cimg, config='-c tessedit_char_whitelist=0123456789x+-÷ --psm 10').strip()
            grid[-1].append(c)
            if verbose:
                print(f"Processed row {i} / {n}, column {j} / {n}: {c}")
            #return cimg
    return grid

### Fetch the Data

In [ ]:
import requests
import io

In [ ]:
url = "https://preview.redd.it/what-is-this-type-of-puzzle-called-and-how-can-i-solve-it-v0-03kkmm081vaf1.png?width=640&crop=smart&auto=webp&s=a2a376c723a96a8a095739291b5a8fe1a75da34d"
puzzle_img = Image.open(io.BytesIO(requests.get(url).content))
#puzzle_img.resize((puzzle_img.size[0]//4, puzzle_img.size[1]//4))
puzzle_img

### Apply the Transcription

In [ ]:
cropped_img = puzzle_img.crop(np.array(puzzle_img.size * 2) * np.array([0.25, 0.15, 0.9, 0.8]))
cropped_img

In [ ]:
n = 4

In [ ]:
grid = transcribe_grid(cropped_img, n, margin=0.25, threshold=64, verbose=True)
grid

### Apply Corrections

In [ ]:
grid = np.array([[int(v) for v in row] for row in grid])
grid[1][0] = -1
grid

## Solving with Z3

### Solver Setup

In [ ]:
import z3

In [ ]:
def create_grid_vars(n):
    row_vars = [z3.Int(f"row_{i}") for i in range(n)]
    col_vars = [z3.Int(f"col_{i}") for i in range(n)]
    return row_vars, col_vars

In [ ]:
def setup_solver(row_vars, col_vars, grid):
    solver = z3.Optimize()
    
    for v in row_vars:
        solver.add(z3.And(0 <= v, v < 3))
    for v in col_vars:
        solver.add(z3.And(0 <= v, v < 3))
        
    objective = solver.maximize(sum( 
        (grid[i][j] - row_vars[i] - col_vars[j]) % 3
        for i in range(n)
        for j in range(n)
    ))
        
    return solver, objective

### Apply to Problem

In [ ]:
row_vars, col_vars = create_grid_vars(n)
row_vars, col_vars

In [ ]:
solver, objective = setup_solver(row_vars, col_vars, grid+1)

In [ ]:
%%time
solver.check()

In [ ]:
model = solver.model()

In [ ]:
objective.value().as_long() - n*n

In [ ]:
for i in range(n):
    print(f"- Press row {i+1} {model.evaluate(row_vars[i]).as_long()} times")
for i in range(n):
    print(f"- Press column {i+1} {model.evaluate(col_vars[i]).as_long()} times")

## Additional Analysis

The total number of states for the board (would actually be brute-forcible):

In [ ]:
3**(n*n)

Total number of distinct button presses (easily brute-forcible):

In [ ]:
3**(2*n)

### Reachable States

As some comments have pointed out, the total number of button presses isn't the total number reachable states, as distinct button presses can lead to the same state (consider pressing no buttons, versus pressing all rows once and all columns twice, and versus vice versa).

In [ ]:
# X = [row_vals, col_vars]
A = np.array([[0] * (2*n) for i in range(n*n)])
for i in range(n):
    for j in range(n):
        A[n * i + j,i] = -1
for i in range(n):
    for j in range(n):
        A[n * j + i,n + i] = -1
A

In [ ]:
b = grid.flatten()+1

We can create a unique identifier for each states with the base 3 number system.

In [ ]:
def reachable_states(A, b):
    states_visited = set()
    for i in range(3**(2*n)):
        X = np.array([(i // 3**j) % 3 for j in range(A.shape[1])])
        state = (A.dot(X) + b) % 3
        uid = sum([d * 3**j for j,d in enumerate(state)])
        states_visited.add(uid)
    return states_visited

In [ ]:
states_visited = reachable_states(A, b)

In [ ]:
len(states_visited)

Interestingly the number of reachable states, $2187$, is $3^7$. Coincidence? Probably not, but I can't be bothered to determine why.

### Perfect Scores

How many states are reachable in the set of perfect scores?

In [ ]:
perfect_states = reachable_states(A, 2*np.ones_like(b))
len(perfect_states)

The same as before. As a fraction of the total number of possible board states this is:

In [ ]:
len(perfect_states) / 3 ** (n*n)

In [ ]:
3 ** (n*n) / len(perfect_states)

Right, this is effectively the same as $3^{9}$

### Change of Being Able to Score 15
u/yagizandro mentioned that the game has an award for reaching a score of 15, and each game starts with a random configuration. What is the probability of having a board upon which a score of 15 is possible?

In [ ]:
nigh_perfect_states = set()
for i in range(n*n):
    nigh_perfect_states |= reachable_states(A, 2*np.ones_like(b) - (np.arange(n*n)==i))

In [ ]:
len(nigh_perfect_states)

This is $2^{4} \cdot 3^{7}$

In [ ]:
len(nigh_perfect_states) / 3 ** (n*n)

In [ ]:
3 ** (n*n) / len(nigh_perfect_states)

What about 15 or a perfect score?

In [ ]:
len(nigh_perfect_states | perfect_states)

This is $17 \cdot 3^{7}$

In [ ]:
len(nigh_perfect_states | perfect_states) / 3 ** (n*n)

In [ ]:
3 ** (n*n) / len(nigh_perfect_states | perfect_states)

This matches the values u/Motor_Raspberry_2150 computed